# <div align="center"><b> YOLOv11 (You Only Look Once)</b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> poetry source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [ ]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

# Recarga automática de módulos en Jupyter Notebook
import os
from pathlib import Path
import random
import time

from PIL import Image
from loguru import logger

# NumPy y utilidades
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# PyTorch
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from ultralytics import YOLO


# Módulos propios
from vision_transformer.plots import plot_confusion_matrix, plot_metric
from vision_transformer.utils import MulticlassAccuracy
from vision_transformer.config import (
    DATA_DIR,
    RANDOM_SEED,
    MODELS_DIR,
    FIGURES_DIR,
    MODEL_DIR_YOLOV11_M,
    METRICS_FILENAME,
    HISTORY_FILENAME,
    PREDICTIONS_FILENAME,
    MLFLOW_URL,
    DATASET_NAME,
    DATASET_VERSION,
    MODEL_NAME_YOLOV11_M,
    PREFECT_URL
)

import mlflow
import requests
import os

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [ ]:
random.seed(RANDOM_SEED)  # Establece la semilla para la reproducibilidad.
TESTING_MODE = False  # Modo de prueba, si es True, se usa un subconjunto pequeño del dataset.

BATCH_SIZE = 64

# Checkpoints a utilizar
MODEL_NAME = MODEL_NAME_YOLOV11_M
MODEL_FOLDER = MODEL_DIR_YOLOV11_M
CHECKPOINT = MODEL_NAME_YOLOV11_M + ".pt"

# MODEL_NAME = MODEL_NAME_SWINV2_BASE
# MODEL_FOLDER = MODELS_DIR_SIWNV2_BASE
# CHECKPOINT = "microsoft/swinv2-base-patch4-window8-256"

# MODEL_NAME = MODEL_NAME_SWINV2_LARGE
# MODEL_FOLDER = MODELS_DIR_SIWNV2_LARGE
# CHECKPOINT = "microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft"

# Optimizaciones
# # torch.set_float32_matmul_precision('highest') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'más alta'.
# torch.set_float32_matmul_precision('high') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'alta'.
# # torch.set_float32_matmul_precision('medium') # Optimización: Establece la precisión de las multiplicaciones de matrices de punto flotante de 32 bits en 'media'.
# # torch.backends.cudnn.benchmark = True # Optimización: Para redes CNN (pero como se usa una capa convolucional, se establece en True).

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Establece el dispositivo.
logger.info(f"Dispositivo actual: {DEVICE}")

# MLflow: Configuración de la URI de seguimiento
try:
    response = requests.get(MLFLOW_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a MLflow establecida correctamente.")
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_URL  # Configura la URI de seguimiento de MLflow.
    os.environ["MLFLOW_EXPERIMENT_NAME"] = CHECKPOINT.replace("/", "_")  # Configura el nombre del experimento de MLflow.
    os.environ["MLFLOW_TAGS"] = '{"model_family": "swinv2"}'
except Exception as e:
    logger.error(f"Error al conectar con MLflow. Tienes levantado el servidor de MLflow?")
    raise SystemExit(f"Error al conectar con MLflow: {e}")

# Prefect: Configuración de Prefect
try:
    response = requests.get(PREFECT_URL)
    response.raise_for_status()  # Verifica si la solicitud fue exitosa.
    logger.success("Conexión a Prefect establecida correctamente.")
except Exception as e:
    logger.error(f"Error al conectar con Prefect. Tienes levantado el servidor de Prefect?")
    raise SystemExit(f"Error al conectar con Prefect: {e}")

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | *Fine-tuning* del modelo swimv2 sobre el dataset EuroSAT                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Análisis exploratorio del proceso de *fine-tuning* del swimv2 sobre el EuroSAT<br/>- *Tarea:* `Clasificación`<br/>- *Modelo*: `yolov11-cls`<br/> - *Dataset*: `EuroSAT` </small>|
<!-- | **Autor** | <small>[Nombre] ([correo]) </small>                                                                                                 | -->

</div>

## Tabla de contenidos
0. [Pasos previos](#pasos-previos)
1. [Carga de datos](#carga-de-datos)
2. [Carga del modelo](#carga-del-modelo)
3. [Preprocesamiento del dataset](#preprocesamiento)
4. [Definicion de las metricas de evaluacion](#metricas)
5. [Entrenamiento del modelo](#entrenamiento)
6. [Resultados](#resultados)

## 0. Pasos previos <a id="pasos-previos"></a>

Ejecuta desde la raíz del proyecto para descargar el dataset EuroSAT:

```bash
python -m vision_transformer.flows.test_flow
```

## 1. Carga de datos <a name="carga-de-datos"></a>

Para el entrenamiento de YOLO no fue necesario la carga del dataset, ya que la utilizando la API de Ultralytics para el entrenamiento del modelo solo necesitamos pasar el directorio del mismo.

## 2. Carga del modelo <a name="carga-del-modelo"></a>

In [3]:
from ultralytics import YOLO
model = YOLO(os.path.join(MODEL_FOLDER, CHECKPOINT))

100%|██████████| 22.4M/22.4M [00:00<00:00, 38.1MB/s]


## 3. Preprocesamiento del dataset <a name="preprocesamiento-del-dataset"></a>

Ultralytics provee automaticamente una serie de aumentaciones que son aplicadas de manera online. En el presente notebook decidimos conservarlas ya que YOLO se utiliza como modelo baseline de comparacion para los modelos basados en Vision Transformers.

## 4. Definición de las métricas de evaluación <a name="metrics"></a>

Para la evaluacion del modelo utilizamos las mismas metricas que aquellas utilizadas para evaluar los modelos basados en Vision Transformers entrenados en los restantes notebooks de el presente proyecto. En este caso, sin embargo, no utilizamos la libreria evaluate de Huggingface, sino que calculamos las metricas de evaluacion utilizando SciKit-Learn sobre las predicciones hechas por el modelo final.

## 5. Entrenamiento del modelo <a name="entrenamiento"></a>

In [ ]:
results = model.train(
    data=os.path.join(DATA_DIR, "processed/EuroSAT_RGB_huggingface"),
    epochs = 50,
    val=True,
    patience = 20,
    device = 0,
    project = MODEL_FOLDER,
    optimizer = "auto",
    degrees = 5
    )

In [ ]:
# Load saved weights
model = YOLO(MODELS_DIR / MODEL_FOLDER / "weights/best.pt")

In [ ]:
logger.info("Evaluando el modelo...")

test_dir = os.path.join(DATA_DIR, "processed/EuroSAT_RGB_huggingface/test")

# Load images and corresponding labels from folders
class_names = sorted(os.listdir(test_dir))  # assumes folder names = class labels
class_to_idx = {name: idx for idx, name in enumerate(class_names)}

images = []
y_true = []

for class_name in class_names:
    class_path = Path(test_dir) / class_name
    for img_file in class_path.glob("*"):
        if img_file.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            images.append(str(img_file))
            y_true.append(class_to_idx[class_name])

# Predict label using the trained model
start_time = time.time()
y_pred = []

for img_path in images:
    result = model.predict(source=img_path, imgsz=224, verbose=False)
    pred_class = result[0].probs.top1
    y_pred.append(pred_class)

runtime = time.time() - start_time

# Compute metrics
eval_accuracy = accuracy_score(y_true, y_pred)
eval_f1 = f1_score(y_true, y_pred, average="weighted")
eval_precision = precision_score(y_true, y_pred, average="weighted")
eval_recall = recall_score(y_true, y_pred, average="weighted")
eval_runtime = runtime
eval_samples_per_second = len(images) / runtime

# Save metrics as a dataframe
metrics = {
    "eval_loss": None,
    "eval_model_preparation_time": None,
    "eval_accuracy": eval_accuracy,
    "eval_f1": eval_f1,
    "eval_precision": eval_precision,
    "eval_recall": eval_recall,
    "eval_runtime": eval_runtime,
    "eval_samples_per_second": eval_samples_per_second,
    "eval_steps_per_second": None,
}

logger.info("Evaluación finalizada. Métricas:")

metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv(MODELS_DIR / MODEL_FOLDER / METRICS_FILENAME, index=False)
metrics_df

2025-06-17 12:17:07.449 | INFO     | __main__:<module>:1 - Evaluando el modelo...
2025-06-17 12:18:03.063 | INFO     | __main__:<module>:51 - Evaluación finalizada. Métricas:


,eval_loss,eval_model_preparation_time,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,None,None,0.981481,0.981528,0.981831,0.981481,55.530458,48.621965,None


In [42]:
# Guardamos las predicciones del modelo en el conjunto de test
results_df = pd.DataFrame(
    {
        "y_true": [model.names[i] for i in y_true],
        "y_pred": [model.names[i] for i in y_pred],
    }
)
results_df.to_csv(MODEL_FOLDER / PREDICTIONS_FILENAME, index=False)

## 6. Resultados <a name="resultados"></a>

In [43]:
history = pd.read_csv(MODEL_FOLDER / HISTORY_FILENAME)
results_df = pd.read_csv(MODEL_FOLDER / PREDICTIONS_FILENAME)
y_true = results_df["y_true"].values
y_pred = results_df["y_pred"].values

In [ ]:
plot_confusion_matrix(
    y_true=y_true,
    y_pred=y_pred,
    labels=sorted(set(y_true)),
    filename=CHECKPOINT.replace("/", "-").replace(".pt", "") + "_confusion_matrix",
    dirpath=FIGURES_DIR / MODEL_NAME,
    show_as_percentaje=True
)

In [ ]:
# Filtramos solo las filas que tienen datos útiles
filtered_history = history.copy()
filtered_history = filtered_history[filtered_history["epoch"].notna()]

# Plot de accuracy
if "metrics/accuracy_top1" in filtered_history.columns:
    plot_metric(
        filtered_history[filtered_history["metrics/accuracy_top1"].notna()],
        x_col="epoch",
        y_cols=["metrics/accuracy_top1"],
        y_labels=["Accuracy de evaluación"],
        title="Accuracy por época",
        filename=CHECKPOINT.replace("/", "-").replace(".pt", "") + "_accuracy_plot",
        dirpath=FIGURES_DIR / MODEL_NAME,
    )